# Edit MSHE setup file using mpyshe

To use MShePy, either:

- add the MIKE Zero installation path/bin/x64 directory to the system/user variable PYTHONPATH (preferred) or
- call sys.path.append("MIKE Zero installation path/bin/x64")

This notebook guides you through how to use the MShePy module, a python package that allows you to interact with and run MIKE SHE simulations from python scripts. Documentation for MShePy is found [here](https://docs.mikepoweredbydhi.com/engine_libraries/MShe/MShePyApi/). MShePy is designed specifically for situations where the users wants to modify the current state based on the current state of other variables in the model, for example, calculating the current evaportranspiration based on the modeled soil moisture at a timestep. This can be done with MShePy in two ways, 

- Incorporating python plugins for use in the MIKE SHE user interface
- Running MIKE SHE simulations from python scripts, where the model may contain python plugins.

Here we will walk through how to use the MShePy plugin to run a model from python with a customizable plugin.

Github examples for how to use MShePy more operationally can be found [here](https://github.com/DHI/MikeShe).

** Note: to run the code in this notebook, python v3.9 and mikeio 2.6.0 must be installed. Though theoretically MShePy should work with python versions up to 3.12, this option currently raises a dll error.

#### *1. Run the model for a few timesteps, extract variable states*
Here we will
- Initialize a MShe model
- Run the model for 200 timesteps
- Print out the min and max precipitation rate at each timestep


In [1]:
from datetime import datetime, timedelta
import mikeio
import sys
import os

os.add_dll_directory(r"C:\Program Files (x86)\DHI\MIKE Zero\2025\bin\x64")
sys.path.append(r"C:\Program Files (x86)\DHI\MIKE Zero\2025\bin\x64")
import MShePy as ms
import subprocess as sp
import numpy as np

MODEL_PATH = r"..\Skjern_Models\Setup\HIP_500m_Skjern_DHI.she"



In [2]:
mikeio.PfsDocument

AttributeError: module 'mikeio' has no attribute 'PfsDocument'

In [3]:
############################################################################################################################################################
# SCRIPT SETUP HERE ALTERED FROM THE DEMO CODE FOUND AT https://github.com/DHI/MikeShe/blob/main/SimulationExecution/execute_stepwise_examples.py
# Alterations:
# - Added function postPrecipRange to demonstrate reading model data from within a plugin
# - Changed model to Skjern
############################################################################################################################################################

#######################################
# Helper functions
#######################################

def pp(setup):
  """Run the MIKE She preprocessor.
  As no python interface is available use the executable and run it as an external process.
  """
  # Get the directory where MShePy was loaded from and use PP from the same installation
  mz_dir = os.path.dirname(ms.__file__)
  pp_exe = os.path.join(mz_dir, "MShe_Preprocessor.exe")
  sp.run([pp_exe, setup])


def enable_plugin(in_path, out_path):
  """Enable using plugins, set the path to the python interpreter and the
  path to this file for plugins - save as copy.
  """
  # Use the same interpreter that is running this script - but we need the dll, not the exe
  py_dir = os.path.dirname(sys.executable) # path to python.exe
  py_dll = f"python{sys.version_info.major}{sys.version_info.minor}.dll"
  py_path = os.path.join(py_dir, py_dll)

  pfs = mikeio.PfsDocument(in_path, unique_keywords=False)
  pfs.MIKESHE_FLOWMODEL.SimSpec.ModelComp.Plugins = 1
  pfs.MIKESHE_FLOWMODEL.Plugins.PyResolve = 1
  # Set paths - special syntax for pfs paths using '|'
  pfs.MIKESHE_FLOWMODEL.Plugins.PyPath = f"|{py_path}|"
  pfs.MIKESHE_FLOWMODEL.Plugins.PluginFileList.PluginFile_1.FILE_NAME = f"|{__file__}|" # this file
  pfs.write(out_path) # save copy


#######################################
# Plugins
#######################################

def postTimeStep():
  """A MIKE She plugin function. No technical problem to put it in the same file as the code
  calling the MIKE She engine, however in a larger project it might be cleaner to separate it.
  """
  ms.wm.log(f"Message from plugin: Time step performed, time now: {ms.wm.currentTime()}")

def postPrecipRange():
  """A MIKE She plugin function for recording the min and max precipitation rate at a time step."""
  (dtStart, dtEnd, values) = ms.wm.getValues(ms.paramTypes.P_RATE)
  ms.wm.log(f"Message from plugin: Precipitation data read for period {dtStart} to {dtEnd}")

  arr = np.array(values[:,:])
  minP = np.nanmin(arr)
  maxP = np.nanmax(arr)
  ms.wm.log(f"-- Min, Max precipitation rate: {minP} mm/h, {maxP} mm/h")

#######################################
# Demo functions
#######################################

def execute_time_steps(setup):
  """Demo: Run the MIKE She water movement engine by taking time steps via the python API."""
  p, e = os.path.splitext(setup)
  setup_plugin = p + "_plugin" + e
  # optional: Enable plugins
  enable_plugin(setup, setup_plugin)

  pp(setup_plugin)
  ms.wm.initialize(setup_plugin)
  performed, dt_hours, first_time = ms.wm.performTimeStep()
  if performed:
    ms.wm.log("Initial time step performed!")
    ms.wm.log(f"  Duration: {dt_hours} h")
    ms.wm.log(f"  New time: {first_time}")
  else:
    ms.wm.log("Failed to execute time step via python")
    return
  dt = timedelta(hours=dt_hours * 3.5)
  next_time = first_time + dt
  ms.wm.log(f"Requesting to run to: {next_time}")
  next_time = ms.wm.runToTime(next_time)
  ms.wm.log(f"Actual new date-time after calling runToTime: {next_time}")
  ms.wm.terminate(True)


if __name__ == "__main__":
  execute_time_steps(MODEL_PATH)

AttributeError: module 'mikeio' has no attribute 'PfsDocument'

In [1]:
import sys
import datetime

sys.path.append(r"C:\Program Files (x86)\DHI\MIKE Zero\2025\bin\x64")
import MShePy as ms


In [2]:
MShePy.wm.initialize(r"..\Skjern_Models\Setup\HIP_500m_Skjern_DHI.she")         # Initialize the model
currentTime = MShePy.wm.currentTime()              # Find the initial timestep
EndTime = currentTime + datetime.timedelta(days=200) # Calculate the end timestep after 200 days
MShePy.wm.runToTime(EndTime)                       # Run the model until the specified timestep
MShePy.wm.terminate(True)                          # Terminate the simulation



In [3]:
currentTime

datetime.datetime(1990, 1, 2, 6, 0)

In [4]:
MShePy.wm.currentTime()

datetime.datetime(1990, 7, 21, 6, 0)

In [5]:
MShePy.wm.simPeriod()

(datetime.datetime(1990, 1, 2, 6, 0), datetime.datetime(2019, 12, 23, 6, 0))

In [6]:
(dtStart, dtEnd, values) = MShePy.wm.getValues(MShePy.paramTypes.P_RATE)
print(f"Precipitation from {dtStart} to {dtEnd}:")
print(values)

Precipitation from 1990-07-20 06:00:00 to 1990-07-21 06:00:00:
MShePy39.Dataset: "precipitation rate", Precipitation Rate (meter/sec), BaseGrid (IN) (OUT), shape: [180, 140], size: 10468


In [ ]:
print(values.shape())
type(values[0:179,0:139])

# convert list of shape [180, 140] to a 2D numpy array
import numpy as np
arr = np.array(values[:,:])
arr.shape
arr

# print non-nan max value
print(np.nanmax(arr))
# print non-nan min value
print(np.nanmin(arr))


[180, 140]
0.0
0.0


: 


#### *2. Create a customizable plugin that updates reference evapotranspiration based on computed soil moisture at each timestep*

https://github.com/DHI/MikeShe/blob/main/SimulationExecution/execute_stepwise_examples.py